In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.contrib.layers

In [3]:
from tensorflow.examples.tutorials.mnist import input_dataini

In [5]:
mnist=input_data.read_data_sets("MNIST_data/")

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [1]:
# generating the discriminator fucntion

def dicriminator(x_image, reuse=None):
    with tf.variable_scope('discriminator', reuse=tf.AUTO_REUSE):
        
        #defining weights
        
        #Weights for first conv layer
        d_w1 = tf.get_variable('d_w1', [5, 5, 1, 32], initializer=tf.truncated_normal_initializer(stddev=0.02))
        #weights for second conv layer
        d_w2 = tf.get_variable('d_w2', [5, 5, 32, 64], initializer=tf.truncated_normal_initializer(stddev=0.02))
        #Weights for fully connected layer
        d_wfc = tf.get_variable('d_wfc', [7*7*64, 5, 1024 ], initializer=tf.truncated_normal_initializer(stddev=0.02))
        #Weights for output layer
        d_wout = tf.get_variable('d_wout', [1024 , 10 ], initializer=tf.truncated_normal_initializer(stddev=0.02))
        
        #defing biases
        
        #bias for first conv layer
        d_b1 = tf.get_variable('d_b1', [32], initializer=tf.constant_initializer(0))
        #bias for second conv layer
        d_b2 = tf.get_variable('d_b2', [64], initializer=tf.constant_initializer(0))
        #bias for fully connected layer
        d_bfc = tf.get_variable('d_bfc', [1024], initializer=tf.constant_initializer(0))
        #bias for output layer
        d_bout = tf.get_variable('d_bout', [10], initializer=tf.constant_initializer(0))
        
        #formation of the neural network layers
        
        #first convolutional layer
        d1 = tf.nn.conv2d(input=x_image, filter=d_w1, strides=[1, 1, 1, 1], padding='SAME')
        d1 = d1 + d_b1
        d1 = tf.nn.relu(d1)
        d1 = tf.nn.avg_pool(d1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        #second convolutonal layer
        d2 = tf.nn.conv2d(input=d1, filter=d_w2, strides=[1, 1, 1, 1], padding='SAME')
        d2 = d2 + d_b2
        d2 = tf.nn.relu(d2)
        d2 = tf.nn.avg_pool(d2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        #fully connected layer
        d_fc = tf.reshape(d2, [-1, 7 * 7 * 64])
        d_fc = tf.matmul(dfc, d_wfc)
        d_fc = d_wfc + d_bfc
        d_fc = tf.nn.relu(d3)
        
        #output layer
        d_out = tf.matmul(d_wfc, d_wout) + d_bout
        
        return d_out

        
                        

In [ ]:
def generator(batch_size, z_dim):
    
    z = tf.truncated_normal([batch_size, z_dim], mean=0, stddev=1, name='z')
    
    #defining weights
    
    #Weights for first deconv layer
    g_w1 = tf.get_variable('g_w1', [z_dim, 3136], dtype=tf.float32,
                           initializer=tf.truncated_normal_initializer(stddev=0.02))
    
    #Weights for second deconv layer
    g_w2 = tf.get_variable('g_w2', [3, 3, 1, z_dim/2], dtype=tf.float32,
                           initializer=tf.truncated_normal_initializer(stddev=0.02))
    
    #Weights for third deconv layer
    g_w3 = tf.get_variable('g_w3', [3, 3, z_dim/2, z_dim/4], dtype=tf.float32, 
                           initializer=tf.truncated_normal_initializer(stddev=0.02))
    
    # Final convolution with one output channel
    g_w4 = tf.get_variable('g_w4', [1, 1, z_dim/4, 1], dtype=tf.float32,
                           initializer=tf.truncated_normal_initializer(stddev=0.02))
    
     
    #Defining biases
    
    #bias for first deconv layer
    g_b1 = tf.get_variable('g_b1', [3136], initializer=tf.truncated_normal_initializer(stddev=0.02))
    #bias for second deconv layer
    g_b2 = tf.get_variable('g_b2', [z_dim/2], initializer=tf.truncated_normal_initializer(stddev=0.02))
    #bias for third deconv layer
    g_b3 = tf.get_variable('g_b3', [z_dim/4], initializer=tf.truncated_normal_initializer(stddev=0.02))
    #bias for final deconv layer
    g_b4 = tf.get_variable('g_b4', [1], initializer=tf.truncated_normal_initializer(stddev=0.02))
    
    #formation of neural network
    
    g1 = tf.matmul(z, g_w1) + g_b1
    g1 = tf.reshape(g1, [-1, 56, 56, 1])
    g1 = tf.contrib.layers.batch_norm(g1, epsilon=1e-5, scope='bn1')
    g1 = tf.nn.relu(g1)
    
    g2 = tf.nn.conv2d(g1, g_w2, strides=[1, 2, 2, 1], padding='SAME')
    g2 = g2 + g_b2
    g2 = tf.contrib.layers.batch_norm(g2, epsilon=1e-5, scope='bn2')
    g2 = tf.nn.relu(g2)
    g2 = tf.image.resize_images(g2, [56, 56])
    
    g3 = tf.nn.conv2d(g2, g_w3, strides=[1, 2, 2, 1], padding='SAME')
    g3 = g3 + g_b3
    g3 = tf.contrib.layers.batch_norm(g3, epsilon=1e-5, scope='bn3')
    g3 = tf.nn.relu(g3)
    g3 = tf.image.resize_images(g3, [56, 56])